Instalación de dependencias

In [1]:
pip install folium

Dependencias

In [2]:
import folium
import json
import csv
import requests
import pandas as pd
import math
import shapely
from shapely.geometry import shape, GeometryCollection, Point, Polygon

In [3]:
vAPIKEY=''
with open('mapboxkeys.csv', mode='r', ) as csv_file:
        csv_reader = csv.DictReader(csv_file, delimiter=',')
        for row in csv_reader:
            vAPIKEY=row["APIKEY"]

In [4]:
with open('seccensales.JSON') as f:
    seccensales_geojson = json.load(f)

In [5]:
def get_coordinates_from_string(street_name, apikey):
    URL = "https://api.mapbox.com/geocoding/v5/mapbox.places/"+str(street_name)+".json?access_token="+apikey
    r = requests.get(url = URL)
    data = r.json()
    return data['features'][0]['geometry']['coordinates']

In [6]:
def get_seccensal(lat,long, infeatures):
  seccensal = 0
  point1 = Point(long, lat)
  for feature in infeatures["features"]:
      poly = Polygon(feature["geometry"]["coordinates"][0])
      if poly.contains(point1) == True:
          seccensal = feature["properties"]["coddistsec"]
          break
  return seccensal

In [7]:
def get_distance_two_locations(lat1, lon1, lat2, lon2):
    rad=math.pi/180
    dlat=lat2-lat1
    dlon=lon2-lon1
    R=6372.795477598
    a=(math.sin(rad*dlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia=2*R*math.asin(math.sqrt(a))
    return distancia

In [8]:
def get_centroide(polygons,element):
  poly = Polygon(polygons[0])
  return poly.centroid.xy[element][0]

In [9]:
def HuffModel(row,Alpha,Beta,num_tienda,total_tiendas):
    operand_A = 0
    sum = 0
    for id_laundry in range(len(vlc_laundry)):
      name_laundry = "dist.lavanderia_"+str(id_laundry)
      name_attrac = "attr.lavanderia_"+str(id_laundry)
      if (num_tienda == id_laundry):
        operand_A +=(pow(row[name_attrac],Alpha)/pow(row[name_laundry],Beta))
      sum +=(pow(row[name_attrac],Alpha)/pow(row[name_laundry],Beta))
    huff_prob = (operand_A / sum)
    return huff_prob

Preparamos un dataframe para plotear resultados

In [10]:
df_output_huff = pd.json_normalize(seccensales_geojson['features'])
df_output_huff['properties.coddistsec'] = df_output_huff['properties.coddistsec'].astype(int)
df_output_huff["centroid_lng"] = df_output_huff.apply(lambda row: get_centroide(row["geometry.coordinates"],0), axis=1)
df_output_huff["centroid_lat"] = df_output_huff.apply(lambda row: get_centroide(row["geometry.coordinates"],1), axis=1)
df_output_huff.drop(['type','properties.coddistrit','properties.codsec','geometry.type','geometry.coordinates'], axis=1, inplace=True)

In [11]:
vlc_laundry = pd.read_csv('lavanderias.csv', sep=',', decimal='.')

Aplicación, Cambiar la dirección sobre la que queremos calcular

In [12]:
street_name = "Perez Galdos 42, Valencia, Valencia"

In [13]:
coordinates = get_coordinates_from_string(street_name, vAPIKEY)

In [14]:
laundry_seccensal = get_seccensal(coordinates[1], coordinates[0], seccensales_geojson)

In [15]:
new_row = {'name': 'new', 'rating': 0, 'user_ratings_total': 0, 'latitude': coordinates[1], 'longitude': coordinates[0], 'type': 'lavanderias', 'coddistsec': laundry_seccensal, 'subtypes': 'laundry'}
vlc_laundry = vlc_laundry.append(new_row, ignore_index=True)

In [17]:
for idx in range(len(seccensales_geojson['features'])):
  seccensales_geojson['features'][idx]['properties']['coddistsec'] = int(seccensales_geojson['features'][idx]['properties']['coddistsec'])

In [18]:
for id_laundry in range(len(vlc_laundry)):
  name_laundry = "dist.lavanderia_"+str(id_laundry)
  df_output_huff[name_laundry] = df_output_huff.apply(lambda row: get_distance_two_locations(vlc_laundry['latitude'][id_laundry],vlc_laundry['longitude'][id_laundry],row['centroid_lat'],row['centroid_lng']), axis=1)


In [19]:
#calculo de la constante atractividad
for id_laundry in range(len(vlc_laundry)):
  name_attrac = "attr.lavanderia_"+str(id_laundry)
  df_output_huff[name_attrac] = 1

In [20]:
#calcular huff model, con atractividad constante = 1.
df_output_huff["Huff_Prob"] = df_output_huff.apply(lambda row: HuffModel(row,1,2,len(vlc_laundry)-1,len(vlc_laundry)), axis=1)

In [21]:
df_output_huff.to_csv('df_output_huff.csv')

In [22]:
map = folium.Map(location=[39.46994829189022, -0.37787440832473984], zoom_start=12)

folium.Choropleth(
    geo_data=seccensales_geojson,
    data=df_output_huff,
    name ='choropleth', 
    columns=["properties.coddistsec", "Huff_Prob"],
    key_on="feature.properties.coddistsec",
    fill_color='YlGnBu', 
    fill_opacity=0.70, 
    line_opacity=1,
    legend_name='Distancia Lavanderia',
    smooth_factor=0
).add_to(map)
folium.Marker(
    location=[coordinates[1], coordinates[0]],
    popup='Nueva Tienda',
).add_to(map)

In [23]:
map